In [10]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

from dotenv import load_dotenv
load_dotenv()

jjkim_key = os.getenv("JINA_API_JJKIM")
obj76_key = os.getenv("JINA_API_OBJECTS76")

from rich import print as pprint

In [11]:

def get_html(url):
    import requests
    from urllib.parse import urlparse

    parsed = urlparse(url)
    if parsed.scheme in ['http', 'https']:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    elif parsed.scheme == 'file':
        with open(parsed.path, 'r', encoding='utf-8') as f:
            return f.read()
    else:
        raise ValueError(f"지원하지 않는 URL 스키마: {parsed.scheme}")

In [14]:
from IPython.display import HTML
HTML( get_html("https://news.hada.io/topic?id=22490") )